In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [2]:
plotting = False

In [3]:
def preprocess_frame(frame):
    blurred_frame = cv2.GaussianBlur(frame, (7, 7), 0)
    gray_frame = cv2.cvtColor(blurred_frame, cv2.COLOR_BGR2GRAY)
    return gray_frame

def detect_car(gray_frame):
    kernel_blackhat = np.ones((15, 15), np.uint8)
    blackhat = cv2.morphologyEx(gray_frame, cv2.MORPH_BLACKHAT, kernel_blackhat)

    _, thresholded = cv2.threshold(blackhat, 50, 255, cv2.THRESH_BINARY)

    kernel_close = np.ones((11, 11), np.uint8)
    closed = cv2.morphologyEx(thresholded, cv2.MORPH_CLOSE, kernel_close, iterations=4)

    contours, _ = cv2.findContours(closed, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    car_contours = []
    for c in contours:
        area = cv2.contourArea(c)
        x, y, w, h = cv2.boundingRect(c)
        if h == 0: 
            continue
        aspect_ratio = w / float(h)
        if area > 2000 and 1.0 < aspect_ratio < 5.0:
            car_contours.append(c)

    if not car_contours:
        return None, closed

    largest_contour = max(car_contours, key=cv2.contourArea)
    return largest_contour, closed

def isolate_car(frame, contour):
    x, y, w, h = cv2.boundingRect(contour)

    margin_x = int(w * 0.05)
    margin_y = int(h * 0.1)

    x_new = max(0, x - margin_x)
    y_new = max(0, y - margin_y)
    w_new = w + 2 * margin_x
    h_new = h + 2 * margin_y
    
    if x_new + w_new > frame.shape[1]:
        w_new = frame.shape[1] - x_new
    if y_new + h_new > frame.shape[0]:
        h_new = frame.shape[0] - y_new

    car_roi = frame[y_new:y_new+h_new, x_new:x_new+w_new]
        
    return car_roi

def find_license_plate(car_roi):
    if car_roi.size == 0:
        return None, None

    gray = cv2.cvtColor(car_roi, cv2.COLOR_BGR2GRAY)
    
    rectKernel = cv2.getStructuringElement(cv2.MORPH_RECT, (13, 5))
    blackhat = cv2.morphologyEx(gray, cv2.MORPH_BLACKHAT, rectKernel)

    sobelx = cv2.Sobel(blackhat, cv2.CV_8U, 1, 0, ksize=3)

    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (17, 3))
    closed = cv2.morphologyEx(sobelx, cv2.MORPH_CLOSE, kernel, iterations=2)

    _, thresh = cv2.threshold(closed, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
    contours, _ = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    for contour in contours:
        rect = cv2.minAreaRect(contour)
        (x, y), (width, height), angle = rect

        if width < height:
            width, height = height, width
            angle += 90
        if abs(angle) > 30 or height == 0:
            continue
        
        aspect_ratio = width / float(height)
        area = cv2.contourArea(contour)

        if 3.7 < aspect_ratio < 6 and 100 < area < 5000:
            box = cv2.boxPoints(rect)
            box = np.intp(box)
            return box, contour

    return None, None

def is_valid_plate_colors(plate_roi, frame_count=None):
    plate_hsv = cv2.cvtColor(plate_roi, cv2.COLOR_BGR2HSV)

    total_pixels = plate_hsv.shape[0] * plate_hsv.shape[1]
    if total_pixels == 0:
        return False

    lower_blue = np.array([100, 150, 50])
    upper_blue = np.array([130, 255, 255])
    lower_white = np.array([0, 0, 100])
    upper_white = np.array([180, 100, 255])
    lower_black = np.array([0, 0, 0])
    upper_black = np.array([180, 255, 50])

    blue_mask = cv2.inRange(plate_hsv, lower_blue, upper_blue)
    white_mask = cv2.inRange(plate_hsv, lower_white, upper_white)
    black_mask = cv2.inRange(plate_hsv, lower_black, upper_black)

    blue_percentage = np.sum(blue_mask > 0) / total_pixels
    white_percentage = np.sum(white_mask > 0) / total_pixels
    black_percentage = np.sum(black_mask > 0) / total_pixels

    if frame_count is not None:
        print(f"Frame {frame_count}: W: {white_percentage:.2f}, B: {black_percentage:.2f}, Blue: {blue_percentage:.2f}")

    if white_percentage > 0.15 and black_percentage > 0.02 and blue_percentage > 0.004 and blue_percentage < 0.2 and black_percentage < 0.25:
        return True
    
    return False


def process_video(video_path):
    cap = cv2.VideoCapture(video_path)
    frame_count = 0
    plate_positions = []

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        frame_count += 1
        # if frame_count % 3 or frame_count < 30:
        #     continue
        gray_frame = preprocess_frame(frame)
        car_contour, processed_image = detect_car(gray_frame)
        
        car_roi_display = np.zeros((1, 1, 3), np.uint8)
        license_plate_roi = np.zeros((1, 1, 3), np.uint8)

        if car_contour is not None:
            car_roi = isolate_car(frame, car_contour)
            if car_roi.size > 0:
                car_roi_display = car_roi.copy()
                plate_box, plate_contour = find_license_plate(car_roi)
                
                license_plate_roi = np.zeros((1, 1, 3), np.uint8)

                if plate_box is not None:
                    x, y, w, h = cv2.boundingRect(plate_contour)
                    potential_plate_roi = car_roi[y:y+h, x:x+w]

                    if is_valid_plate_colors(potential_plate_roi): # frame_count
                        license_plate_roi = potential_plate_roi
                        cv2.drawContours(car_roi_display, [plate_box], 0, (0, 255, 0), 2)
                        plate_positions.append((frame_count, plate_box))
                        
        if plotting:
            fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15, 5))
            fig.suptitle(f"Frame {frame_count}")

            ax1.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
            ax1.set_title('Original Frame')
            ax1.axis('off')

            if car_roi_display.size > 1:
                ax2.imshow(cv2.cvtColor(car_roi_display, cv2.COLOR_BGR2RGB))
            ax2.set_title('Isolated Car with Plate Contour')
            ax2.axis('off')

            if license_plate_roi.size > 1:
                ax3.imshow(cv2.cvtColor(license_plate_roi, cv2.COLOR_BGR2RGB))
            ax3.set_title('Detected License Plate')
            ax3.axis('off')
            
            plt.show()

    cap.release()
    return plate_positions

In [4]:
# plate_positions = process_video('video/20251011_134750.mp4')
plate_positions = process_video('video/20251011_134803.mp4')

In [5]:
if plate_positions:
    first_frame_offset = plate_positions[0][0]
    with open("plate_positions.txt", 'w') as f:
        for frame_count, box in plate_positions:
            normalized_frame_count = frame_count - first_frame_offset
            box_str = ",".join([f"[{p[0]},{p[1]}]" for p in box])
            f.write(f"{normalized_frame_count}: {box_str}\n")
else:
    print("No Plates")